### Load the packages

In [1]:
# Do not modify import block, all necessary imports are included
# Our regular libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import xarray as xr
%matplotlib inline

# for the sampling and statistical comparison
import numpy as np
from scipy import stats

# for plotting
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# for plotting
import time
import warnings
warnings.filterwarnings('ignore')

#import "/scratch/project_2000789/muramarg/miniconda3/lib/python3.10/site-packages/gsw"
import sys
sys.path.append('/scratch/project_2000789/muramarg/miniconda3/lib/python3.10/site-packages/')
import gsw

### Load the data

In [2]:
# write these to files? does this save time
xgrid = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/xgrid.txt",sep=",")
ygrid = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/ygrid.txt",sep=",")
depth = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/depth.txt",sep=",")

xgrid = xgrid.reshape((35040,6586))
ygrid = ygrid.reshape((35040,6586))
depth = depth.reshape((35040,6586))

In [3]:
temp = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/temp.txt",sep=",")
salt = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/salt.txt",sep=",")
salt = salt.reshape((35040,6586))
temp = temp.reshape((35040,6586))

In [4]:
wmt = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/mass_wmt.txt",sep=",")
wmt = wmt.reshape((35040,-1))

In [5]:
# convert the waom values to gsw values
# convert depth to sea pressure
long = 73.5089
lat = -66.8245
p = gsw.p_from_z(z=depth,lat=-66.8245)


# convert the absolute salinity from practical salinity
SA = gsw.SA_from_SP(salt,p,long,lat)    # absolute salinity from practical salinity

# convert potential T to conservative T
CT = gsw.CT_from_pt(SA,temp)   # conservative T from potential T

# find the potential density from SA, CT, p
rho_pot = gsw.rho(SA,CT,p) - 1000

In [6]:
# convert the absolute salinity from practical salinity
# dSA = SA[-1]-SA[0]
dSA = salt[-1]-salt[0]

# convert potential T to conservative T
# dCT = CT[-1]-CT[0]
dCT = temp[-1]-temp[0]

# find the potential density from SA, CT, p
drho_pot = rho_pot[-1]-rho_pot[0]

df = pd.DataFrame()
df["dT"] = dCT
df["dsat"] = dSA
df["drho"] = drho_pot
df

,dT,dsat,drho
0,-0.012908,0.228996,0.412905
1,0.480041,0.213917,1.122429
2,0.459540,0.221115,1.208951
3,-0.131968,0.027908,0.044213
4,-0.182100,0.168007,0.262203
...,...,...,...
6581,-0.810600,-0.071541,1.949936
6582,-1.049942,0.006630,2.399629
6583,-0.033861,-0.040905,1.748677
6584,-0.353143,-0.136509,0.035242


In [7]:
# load with 4 groups
group1 = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/GroupFiles/group1_4grps.txt",sep=",")
group2 = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/GroupFiles/group2_4grps.txt",sep=",")
group3 = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/GroupFiles/group3_4grps.txt",sep=",")
group4 = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/GroupFiles/group4_4grps.txt",sep=",")

group1 = group1.tolist()
group1 = [int(x) for x in group1]
group2 = group2.tolist()
group2 = [int(x) for x in group2]
group3 = group3.tolist()
group3 = [int(x) for x in group3]
group4 = group4.tolist()
group4 = [int(x) for x in group4]

groups = np.array([group1,group2,group3,group4],dtype=object)

# Kolmogorov-Smirnov
### Loop through and save the samples

In [ ]:
firstsamp = salt[:,group4].flatten()
firstsamp = np.append(firstsamp,temp[:,group4].flatten())

reslst = np.array([])
for i in range(len(groups)):
    
    secondsamp = salt[:,groups[i]].flatten()
    secondsamp = np.append(secondsamp,temp[:,groups[i]].flatten())
    
    res = stats.ks_2samp(firstsamp, secondsamp)
    reslst = np.append(reslst,res)
    
    firstsamp = secondsamp.copy()
    print(res)

# Anderson-Darnling

In [8]:
# example of AD
from scipy.stats import anderson
rng = np.random.default_rng()
data = rng.random(size=35)
res = anderson(data)
# res.statistic
# res.critical_values
# res.significance_level

# Sammon mapping

In [9]:
def sammon(x, n, display = 2, inputdist = 'raw', maxhalves = 20, maxiter = 500, tolfun = 1e-9, init = 'default'):

    import numpy as np 
    from scipy.spatial.distance import cdist

    """Perform Sammon mapping on dataset x

    y = sammon(x) applies the Sammon nonlinear mapping procedure on
    multivariate data x, where each row represents a pattern and each column
    represents a feature.  On completion, y contains the corresponding
    co-ordinates of each point on the map.  By default, a two-dimensional
    map is created.  Note if x contains any duplicated rows, SAMMON will
    fail (ungracefully). 

    [y,E] = sammon(x) also returns the value of the cost function in E (i.e.
    the stress of the mapping).

    An N-dimensional output map is generated by y = sammon(x,n) .

    A set of optimisation options can be specified using optional
    arguments, y = sammon(x,n,[OPTS]):

       maxiter        - maximum number of iterations
       tolfun         - relative tolerance on objective function
       maxhalves      - maximum number of step halvings
       input          - {'raw','distance'} if set to 'distance', X is 
                        interpreted as a matrix of pairwise distances.
       display        - 0 to 2. 0 least verbose, 2 max verbose.
       init           - {'pca', 'cmdscale', random', 'default'}
                        default is 'pca' if input is 'raw', 
                        'msdcale' if input is 'distance'

    The default options are retrieved by calling sammon(x) with no
    parameters.

    File        : sammon.py
    Date        : 18 April 2014
    Authors     : Tom J. Pollard (tom.pollard.11@ucl.ac.uk)
                : Ported from MATLAB implementation by 
                  Gavin C. Cawley and Nicola L. C. Talbot

    Description : Simple python implementation of Sammon's non-linear
                  mapping algorithm [1].

    References  : [1] Sammon, John W. Jr., "A Nonlinear Mapping for Data
                  Structure Analysis", IEEE Transactions on Computers,
                  vol. C-18, no. 5, pp 401-409, May 1969.

    Copyright   : (c) Dr Gavin C. Cawley, November 2007.

    This program is free software; you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation; either version 2 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program; if not, write to the Free Software
    Foundation, Inc., 59 Temple Place, Suite 330, Boston, MA 02111-1307 USA

    """

    # Create distance matrix unless given by parameters
    if inputdist == 'distance':
        D = x
        if init == 'default':
            init = 'cmdscale'
    else:
        D = cdist(x, x)
        if init == 'default':
            init = 'pca'

    if inputdist == 'distance' and init == 'pca':
        raise ValueError("Cannot use init == 'pca' when inputdist == 'distance'")

    if np.count_nonzero(np.diagonal(D)) > 0:
        raise ValueError("The diagonal of the dissimilarity matrix must be zero")

    # Remaining initialisation
    N = x.shape[0]
    scale = 0.5 / D.sum()
    D = D + np.eye(N)     

    if np.count_nonzero(D<=0) > 0:
        raise ValueError("Off-diagonal dissimilarities must be strictly positive")   

    Dinv = 1 / D
    if init == 'pca':
        [UU,DD,_] = np.linalg.svd(x)
        y = UU[:,:n]*DD[:n] 
    elif init == 'cmdscale':
        from cmdscale import cmdscale
        y,e = cmdscale(D)
        y = y[:,:n]
    else:
        y = np.random.normal(0.0,1.0,[N,n])
    one = np.ones([N,n])
    d = cdist(y,y) + np.eye(N)
    dinv = 1. / d
    delta = D-d 
    E = ((delta**2)*Dinv).sum() 

    # Get on with it
    for i in range(maxiter):

        # Compute gradient, Hessian and search direction (note it is actually
        # 1/4 of the gradient and Hessian, but the step size is just the ratio
        # of the gradient and the diagonal of the Hessian so it doesn't
        # matter).
        delta = dinv - Dinv
        deltaone = np.dot(delta,one)
        g = np.dot(delta,y) - (y * deltaone)
        dinv3 = dinv ** 3
        y2 = y ** 2
        H = np.dot(dinv3,y2) - deltaone - np.dot(2,y) * np.dot(dinv3,y) + y2 * np.dot(dinv3,one)
        s = -g.flatten(order='F') / np.abs(H.flatten(order='F'))
        y_old    = y

        # Use step-halving procedure to ensure progress is made
        for j in range(maxhalves):
            s_reshape = np.reshape(s, (-1,n),order='F')
            y = y_old + s_reshape
            d = cdist(y, y) + np.eye(N)
            dinv = 1 / d
            delta = D - d
            E_new = ((delta**2)*Dinv).sum()
            if E_new < E:
                break
            else:
                s = 0.5*s

        # Bomb out if too many halving steps are required
        if j == maxhalves-1:
            print('Warning: maxhalves exceeded. Sammon mapping may not converge...')

        # Evaluate termination criterion
        if abs((E - E_new) / E) < tolfun:
            if display:
                print('TolFun exceeded: Optimisation terminated')
            break

        # Report progress
        E = E_new
        if display > 1:
            print('epoch = %d : E = %12.10f'% (i+1, E * scale))

    if i == maxiter-1:
        print('Warning: maxiter exceeded. Sammon mapping may not have converged...')

    # Fiddle stress to match the original Sammon paper
    E = E * scale
    
    return [y,E]

In [10]:
# create flattened arrays
sample1 = salt[:,group1].flatten()
sample1 = np.append(sample1,temp[:,group1].flatten())

sample2 = salt[:,group2].flatten()
sample2 = np.append(sample2,temp[:,group2].flatten())

sample3 = salt[:,group3].flatten()
sample3 = np.append(sample3,temp[:,group3].flatten())

sample4 = salt[:,group4].flatten()
sample4 = np.append(sample4,temp[:,group4].flatten())

In [11]:
myx = np.full((308772480,4),np.nan)
myx[:,0] = sample1
myx[:len(sample2),1] = sample2
myx[:len(sample3),2] = sample3
myx[:len(sample4),3] = sample4

In [12]:
# make an array of the group label, this will be our target
target = np.full(salt[0].shape,np.nan)
gn = 1
for group in groups:
    for i in group:
        target[i] = gn
    gn += 1

In [13]:
saltT = salt.T
tempT = temp.T

In [ ]:
allts = np.concatenate((saltT,tempT))
allts.shape

In [ ]:
target2 = np.concatenate((target,target))

In [ ]:
target2 = target2.astype(int)

In [ ]:
target2.shape

In [ ]:
# target = target2
names = np.array(["Group 1","Group 2", "Group 3","Group 4"])
x = saltT
# Run the Sammon projection
[y,E] = sammon(x, 2)

In [ ]:
# Plot
plt.scatter(y[target ==1, 0], y[target ==1, 1], s=20, c='r', marker='o',label=names[0])
plt.scatter(y[target ==2, 0], y[target ==2, 1], s=20, c='b', marker='D',label=names[1])
plt.scatter(y[target ==3, 0], y[target ==3, 1], s=20, c='y', marker='v',label=names[2])
plt.scatter(y[target ==4, 0], y[target ==4, 1], s=20, c='g', marker='*',label=names[3])
plt.title("Sammon mapping of salt")
plt.legend(loc=2)

plt.savefig("/scratch/project_2000789/muramarg/floats_WAOM/meeting_6_26/sammon_salt.png", dpi=300)

In [ ]:
target

In [ ]:
# target = target2
names = np.array(["Group 1","Group 2", "Group 3","Group 4"])
x2 = tempT
# Run the Sammon projection
[y2,E2] = sammon(x2, 2)

epoch = 1 : E = 0.0322054247
epoch = 2 : E = 0.0320296141
epoch = 3 : E = 0.0319550968
epoch = 4 : E = 0.0319519500
epoch = 5 : E = 0.0310008804
epoch = 6 : E = 0.0308858128
epoch = 7 : E = 0.0308831332
epoch = 8 : E = 0.0307774668
epoch = 9 : E = 0.0306755552
epoch = 10 : E = 0.0306721355
epoch = 11 : E = 0.0299948996
epoch = 12 : E = 0.0299718074
epoch = 13 : E = 0.0297268316
epoch = 14 : E = 0.0297233872
epoch = 15 : E = 0.0293229095
epoch = 16 : E = 0.0292550494
epoch = 17 : E = 0.0292474187
epoch = 18 : E = 0.0291280853
epoch = 19 : E = 0.0289856383
epoch = 20 : E = 0.0289818729
epoch = 21 : E = 0.0289714781
epoch = 22 : E = 0.0286122100
epoch = 23 : E = 0.0285801914
epoch = 24 : E = 0.0285572111
epoch = 25 : E = 0.0285089402


In [ ]:
# Plot
plt.scatter(y2[target ==1, 0], y2[target ==1, 1], s=20, c='r', marker='o',label=names[0])
plt.scatter(y2[target ==2, 0], y2[target ==2, 1], s=20, c='b', marker='D',label=names[1])
plt.scatter(y2[target ==3, 0], y2[target ==3, 1], s=20, c='y', marker='v',label=names[2])
plt.scatter(y2[target ==4, 0], y2[target ==4, 1], s=20, c='g', marker='*',label=names[3])
plt.legend(loc=2)

plt.savefig("/scratch/project_2000789/muramarg/floats_WAOM/meeting_6_26/sammon_temp.png", dpi=300)